In [36]:
import os 
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [4]:
data = pd.read_csv('./data/bank-additional-full 2.csv', sep=';')

In [5]:
data

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,retired,married,professional.course,no,yes,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes
41184,46,blue-collar,married,professional.course,no,no,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41185,56,retired,married,university.degree,no,yes,no,cellular,nov,fri,...,2,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41186,44,technician,married,professional.course,no,no,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes


In [9]:
data = pd.get_dummies(data, columns = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'poutcome'])

In [10]:
data['id'] = range(len(data))

In [11]:
len(data)

41188

In [13]:
train = data.sample(30000, replace=False, random_state=2020).reset_index().drop(['index'], axis=1)
test = data.loc[~data['id'].isin(train['id'])].reset_index().drop(['index'], axis=1)

# RandomForest

In [15]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators = 500, min_samples_split = 10)

In [18]:
input_var = ['age', 'duration', 'campaign', 'pdays', 'previous', 'emp.var.rate',
       'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed', 
       'job_admin.', 'job_blue-collar', 'job_entrepreneur', 'job_housemaid',
       'job_management', 'job_retired', 'job_self-employed', 'job_services',
       'job_student', 'job_technician', 'job_unemployed', 'job_unknown',
       'marital_divorced', 'marital_married', 'marital_single',
       'marital_unknown', 'education_basic.4y', 'education_basic.6y',
       'education_basic.9y', 'education_high.school', 'education_illiterate',
       'education_professional.course', 'education_university.degree',
       'education_unknown', 'default_no', 'default_unknown', 'default_yes',
       'housing_no', 'housing_unknown', 'housing_yes', 'loan_no',
       'loan_unknown', 'loan_yes', 'contact_cellular', 'contact_telephone',
       'month_apr', 'month_aug', 'month_dec', 'month_jul', 'month_jun',
       'month_mar', 'month_may', 'month_nov', 'month_oct', 'month_sep',
       'day_of_week_fri', 'day_of_week_mon', 'day_of_week_thu',
       'day_of_week_tue', 'day_of_week_wed', 'poutcome_failure',
       'poutcome_nonexistent', 'poutcome_success', 'id']

In [19]:
rf.fit(train[input_var], train['y'])

RandomForestClassifier(min_samples_split=10, n_estimators=500)

In [21]:
predictions =rf.predict(test[input_var])

In [22]:
test['pred'] = predictions

In [24]:
(test['pred'] == test['y']).mean()

0.9140150160886664

# 의사결정나무와 비교

In [25]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(min_samples_split=10)

In [26]:
dt.fit(train[input_var], train['y'])

DecisionTreeClassifier(min_samples_split=10)

In [27]:
test['pred'] = dt.predict(test[input_var])

In [28]:
(test['pred'] == test['y']).mean()

0.895244905255631

# Xgboost

In [29]:
from xgboost import XGBClassifier

In [30]:
xgb = XGBClassifier(n_estimators = 300, learning_rate=0.1)

In [31]:
xgb.fit(train[input_var], train['y'])

/Users/mingggkeee/miniforge3/envs/mingggkeee/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[18:47:39] WARNING: /private/var/folders/01/49f3f2tj4lld87fzpx101_wc0000gn/T/pip-install-a0pa1235/xgboost_6524f13849a7456cb0fa8654373e0e99/build/temp.macosx-11.0-arm64-3.8/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=300, n_jobs=8, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [32]:
predictions = xgb.predict(test[input_var])

In [33]:
test['pred'] = predictions

In [34]:
(test['pred'] == test['y']).mean()

0.914730067929925

<strong> 최적의 n_estimators 찾기 </strong>

In [37]:
for n in [100, 200, 300, 400, 500, 600, 700, 800, 900]:
    xgb = XGBClassifier(n_estimators=n, learning_rate=0.1)
    xgb.fit(train[input_var], train['y'])
    predictions = xgb.predict(test[input_var])
    test['pred'] = predictions
    print((test['pred'] == test['y']).mean())

[18:51:29] WARNING: /private/var/folders/01/49f3f2tj4lld87fzpx101_wc0000gn/T/pip-install-a0pa1235/xgboost_6524f13849a7456cb0fa8654373e0e99/build/temp.macosx-11.0-arm64-3.8/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0.9160707901322845
[18:51:31] WARNING: /private/var/folders/01/49f3f2tj4lld87fzpx101_wc0000gn/T/pip-install-a0pa1235/xgboost_6524f13849a7456cb0fa8654373e0e99/build/temp.macosx-11.0-arm64-3.8/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0.9149088308902396
[18:51:35] WARNING: /private/var/folders/01/49f3f2tj4lld87fzpx101_wc0000gn/T/pip-install-a0pa1235/xgboost_6524f13849a7456cb0fa8654373e0e99/build